In [2]:
import os
import dash
from dash import dcc, html
import pandas as pd
import base64
import subprocess
import psycopg2

# 確定目錄設置正確，使用相對路徑設置文件路徑
current_dir = os.getcwd()
target_dir = os.path.join(current_dir, 'dash_web')

# 設置資料庫連線
conn = psycopg2.connect(
    dbname='tvdi_postgresql_etik',   
    user='tvdi_postgresql_etik_user',  
    password='M0pQwAscutSJOVqMbHc2KNpD7bdprn2i',  
    host='dpg-cpscveaj1k6c738l7of0-a.singapore-postgres.render.com',  
    port='5432'  
)

# 假設您的表格名稱是 dash_web，以及要選取的欄位
query = "SELECT * FROM dash_web"

# 使用 Pandas 讀取資料
data = pd.read_sql_query(query, conn)

# 關閉資料庫連線
conn.close()

# 設置顯示圖像的函數
def display_image(image_path):
    encoded_image = base64.b64encode(open(image_path, 'rb').read())
    return f'data:image/png;base64,{encoded_image.decode()}'

# 假設 button_texts 是一個列表，包含按鈕的文本和圖像路徑
button_texts = [
    ("統計摘要", os.path.join(target_dir, 'data.png')),
    ("盒鬚圖", os.path.join(target_dir, 'boxplot_no_outliers.png')),
    ("每日平均日照時數", os.path.join(target_dir, 'line_H.png')),
    ("平均日照時數常態分佈", os.path.join(target_dir, 'normaldistribution_H.png')),
    ("每日平均太陽輻射量", os.path.join(target_dir, 'line_R.png')),
    ("平均日射量常態分佈", os.path.join(target_dir, 'normaldistribution_R.png')),
    ("熱力圖", os.path.join(target_dir, 'heatmap.png')),
    ("線性回歸", os.path.join(target_dir, 'linear_regression.png')),
]

# 創建 Dash 應用
app = dash.Dash(__name__)

# 創建主要的 layout
app.layout = html.Div([
    html.H1("資料庫查看器與圖片"),
    
    # 使用 DataTable 來顯示資料
    dcc.Graph(
        id='tree',
        figure={
            'data': [
                {
                    'type': 'table',
                    'header': {
                        'values': data.columns,
                        'align': 'center',
                        'fill': {'color': '#0074D9'},
                        'font': {'color': 'white', 'size': 14}
                    },
                    'cells': {
                        'values': data.values.tolist(),
                        'align': 'center'
                    }
                }
            ],
            'layout': {
                'height': 400,
                'margin': {'t': 50, 'l': 10, 'r': 10, 'b': 10}
            }
        }
    ),
    
    # 創建按鈕和圖像顯示框
    html.Div([
        html.Div([
            html.Button(text, id=f'button-{idx}', n_clicks=0) for idx, (text, _) in enumerate(button_texts)
        ]),
        html.Div(id='image-display')
    ], style={'width': '50%', 'display': 'inline-block'}),
    
    # 添加計算器按鈕
    html.Button("太陽能系統評估計算", id='calc-button', n_clicks=0),
    html.Div(id='calculator-output')
])

# 回調函數來更新圖像和計算器部分
@app.callback(
    dash.dependencies.Output('image-display', 'children'),
    [dash.dependencies.Input(f'button-{idx}', 'n_clicks') for idx, _ in enumerate(button_texts)]
)
def update_image(*args):
    ctx = dash.callback_context
    if not ctx.triggered:
        button_id = 'button-0'
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if button_id.startswith('button-'):
        idx = int(button_id.split('-')[1])
        path = button_texts[idx][1]
        return html.Img(src=display_image(path), style={'width': '80%', 'height': '80%'})

@app.callback(
    dash.dependencies.Output('calculator-output', 'children'),
    [dash.dependencies.Input('calc-button', 'n_clicks')]
)
def update_calculator(n_clicks):
    if n_clicks > 0:
        # 啟動計算器應用
        subprocess.Popen(["python", "calculator.py"])
        return html.Div("正在啟動計算器...")

if __name__ == '__main__':
    app.run_server(host='127.0.0.1', port=8050, debug=False)


C:\Users\user\AppData\Local\Temp\ipykernel_12880\834608686.py:26: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



[2024-07-23 10:25:39,979] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\user\miniconda3\envs\myenv\lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\user\miniconda3\envs\myenv\lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\user\miniconda3\envs\myenv\lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\user\miniconda3\envs\myenv\lib\site-packages\flask\app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "c:\Users\user\miniconda3\envs\myenv\lib\site-packages\dash\dash.py", line 1373, in dispatch
    ctx.run(
  File "c:\Users\user\miniconda3\envs\myenv\lib\site-packages\dash\_callback.py", line 465, in add_context
    ou